In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import time
import pandas as pd
import json
import datetime
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType,LongType, ArrayType, MapType, DoubleType, BooleanType
from pyspark.sql.functions import array, create_map, struct
from pyspark.sql import Row
from pyspark.sql.functions import from_json, col
from pyspark.sql import functions as F
from pyspark.sql.functions import col 
from pyspark.sql.functions import lit
from delta.tables import *
from datetime import datetime, timedelta
from pyspark.sql import functions as F

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 3, Finished, Available, Finished)

In [101]:
def aggr_table_upsert(table_name, source_df,insert_list,update_list):
    '''
    target_table과 현재 source dataframe 비교하여 upsert
    '''
    target_table = DeltaTable.forName(spark, table_name)
    target_table.alias('aggr_table') \
  .merge(
    source_df.alias('updates'),
    'aggr_table.siteId = updates.siteId and aggr_table.aggr_dt = updates.aggr_dt'
  ) \
  .whenMatchedUpdate(set = update_list
  ) \
  .whenNotMatchedInsert(values = insert_list
  ) \
  .execute()

def make_update_list(df,pk_list):
    columns=list(df.columns)
    insert_list=dict()
    update_list=dict()
    for col in columns:
        insert_list[col]=f"updates.{col}"
    for pk in pk_list:
        columns.remove(pk)
    for col in columns:
        update_list[col]=f"updates.{col}"
    return insert_list,update_list

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 103, Finished, Available, Finished)

In [3]:
schedule_time=""

StatementMeta(, , , Waiting, , Waiting)

In [124]:
# schedule_time=datetime.now()

# start_time=schedule_time-timedelta(minutes=20)

# end_time=schedule_time-timedelta(minutes=15)
# start_time_text=start_time.strftime("%Y-%m-%dT%H:%M:%S")
# end_time_text=end_time.strftime("%Y-%m-%dT%H:%M:%S")
# start_time_text='2024-08-21T05:40:00'
# end_time_text='2024-08-21T05:45:00'

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 126, Finished, Available, Finished)

In [21]:
# start_time_text=schedule_time.strftime("%Y-%m-%dT%H:%M:%S")

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 23, Finished, Available, Finished)

In [33]:
schedule_time=schedule_time.split('.')[0].replace('T',' ')
schedule_time_replaced=datetime.strptime(schedule_time, '%Y-%m-%d %H:%M:%S')
schedule_time_replaced=schedule_time_replaced.replace(second=0)
start_time=schedule_time_replaced-timedelta(minutes=10)

end_time=schedule_time_replaced-timedelta(minutes=5)

start_time_text=start_time.strftime("%Y-%m-%dT%H:%M:%S")
end_time_text=end_time.strftime("%Y-%m-%dT%H:%M:%S")
print(f'start_time_text:{start_time_text}')
print(f'end_time_text:{end_time_text}')

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 35, Finished, Available, Finished)

NameError: name 'schedule_time_replaced' is not defined

In [137]:
# df_ag = spark.sql(f"select * from assetRegistry_dev.telemetry_5min_aggr_table order by aggr_dt desc")

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 139, Finished, Available, Finished)

In [139]:
# display(df_ag)

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 141, Finished, Available, Finished)

In [110]:
# df_ag = spark.sql(f"delete from assetRegistry_dev.telemetry_5min_aggr_table where aggr_dt='NULL'")

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 112, Finished, Available, Finished)

In [125]:
#         table_list_processed=get_table_name_tuple(df_filtered)
df_state = spark.sql(f"select * from assetRegistry_dev.telemetry_1min_bronze where timestamp<='{end_time_text}' and  timestamp>='{start_time_text}' ")

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 127, Finished, Available, Finished)

In [126]:
display(df_state)

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 128, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 75ff7868-ae99-419f-bf20-9418d79116e4)

In [127]:
df_state_aggr=df_state.groupBy('siteId').agg(F.avg("acCurrent").alias("acCurrent"),F.avg("acGridCharging").alias("acGridCharging"),F.avg("acGridChargingTotal").alias("acGridChargingTotal"),F.avg("acPower").alias("acPower_avg"),F.avg("acVoltage").alias("acVoltage") ,F.max("assignedTOUProfile").alias("assignedTOUProfile"),F.avg("backupEventEnergyDischarged").alias("backupEventEnergyDischarged"),F.max("backupEventFailure").alias("backupEventFailure"),F.max("backupEventRunTime").alias("backupEventRunTime"),F.avg("batteryChargeEnergy").alias("batteryChargeEnergy"),F.avg("batteryChargePower").alias("batteryChargePower"),F.avg("batteryDischargeEnergy").alias("batteryDischargeEnergy"),F.avg("batteryDischargePower").alias("batteryDischargePower"),F.avg("batteryLifetimeChargedEnergy").alias("batteryLifetimeChargedEnergy"),F.avg("batteryLifetimeDischargedEnergy").alias("batteryLifetimeDischargedEnergy"),F.max("batteryMode").alias("batteryMode"),F.max("batteryState").alias("batteryState"),F.max("communicationStatus").alias("communicationStatus"),F.avg("dcCurrent").alias("dcCurrent"),F.avg("dcEnergy").alias("dcEnergy"),F.avg("dcPower").alias("dcPower"),F.avg("dcVoltage").alias("dcVoltage"),F.max("energyReadingDuration").alias("energyReadingDuration"),F.avg("exportEnergy").alias("exportEnergy"),F.avg("exportPower").alias("exportPower"),F.max("gridConnectedStatus").alias("gridConnectedStatus"),F.avg("gridFrequency").alias("gridFrequency"),F.max("gridOutageEventFlag").alias("gridOutageEventFlag"),F.avg("gridVoltage").alias("gridVoltage"),F.avg("importEnergy").alias("importEnergy"),F.avg("importPower").alias("importPower"),F.max("inverterMode").alias("inverterMode"),F.avg("loadEnergy").alias("loadEnergy"))
df_state_aggr=df_state_aggr.withColumn('aggr_dt',  lit(start_time_text))
pk_list=['aggr_dt','siteId']
insert_list,update_list=make_update_list(df_state_aggr,pk_list)
aggr_table='telemetry_5min_aggr_table'
display(df_state_aggr)

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 129, Finished, Available, Finished)

In [133]:
if df_state_aggr.count()>0:

    if  spark.catalog.tableExists(f"assetRegistry_dev.{aggr_table}")==False:
        df_state_aggr.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(aggr_table)
    else:
        #df_state.write.mode("append").format("delta").insertInto(aggr_table)
        aggr_table_upsert(aggr_table, df_state_aggr,insert_list,update_list)
        print('date upsert finished')

StatementMeta(, 40781370-d4af-4e62-9d5b-e315cb63d5e7, 135, Finished, Available, Finished)

date upsert finished


In [ ]:
from notebookutils import mssparkutils  

mssparkutils.notebook.exit(schedule_time)  